In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import datasets
from torchtext.legacy import data
import random

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
path = '/content/gdrive/My Drive/IIPL/'

In [4]:
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [5]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

In [6]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

cpu와 cuda 중 다음 기기로 학습함: cpu


In [7]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

In [15]:
from torchtext.legacy.data import TabularDataset

In [16]:
import pandas as pd

In [17]:
df = pd.read_csv(path+"IMDB_Dataset.csv", encoding = 'latin1')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
dataset = TabularDataset(path = path+"IMDB_Dataset.csv", format='csv', fields=[('text', TEXT), ('label', LABEL)], skip_header=True)
trainset, testset = dataset.split(split_ratio=0.8)


In [19]:
# train_df.to_csv("train_data.csv", index=False)
# test_df.to_csv("test_data.csv", index=False)

In [20]:
# trainset, testset = TabularDataset.splits(
#         path='.', train='train_data.csv', test='test_data.csv', format='csv',
#         fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

In [21]:
print('trainset의 구성 요소 출력 : ', trainset.fields)


trainset의 구성 요소 출력 :  {'text': <torchtext.legacy.data.field.Field object at 0x7f80f7e0f150>, 'label': <torchtext.legacy.data.field.Field object at 0x7f80f7e0f850>}


In [22]:
print(vars(trainset[0]))


{'text': ['i', 'was', 'watching', 'this', 'movie', 'and', 'getting', 'increasingly', 'bored', 'with', 'the', 'silly', 'plot', 'that', 'was', 'going', 'nowhere,', 'when', 'suddenly,', 'the', 'story', 'takes', 'a', 'surreal', 'turn', 'for', 'the', 'worse', 'and', 'has', 'an', 'actor', 'playing', 'herself.', 'oh', 'how', 'i', 'guffawed.', 'because', "it's", 'sooooo', 'funny,', "isn't", 'it?', 'we', 'know', 'julia', 'roberts', 'is', 'playing', 'the', 'character', 'of', 'tess,', 'and', 'here', 'they', 'are,', 'in', 'the', 'film,', 'cracking', 'the', 'joke', 'that', 'the', 'character', 'of', 'tess', 'looks', 'a', 'bit', 'like', 'julia', 'roberts.', 'so', 'julia', 'plays', 'someone', 'impersonating', 'julia.', 'how', 'well', 'she', 'does', 'this,', "we'll", 'never', 'know,', 'because', '99.999%', 'of', 'the', 'audience', "don't", 'actually', 'know', 'julia', 'roberts', 'personally', '(and', 'reading', 'about', 'her', 'in', 'hello', 'magazine', "doesn't", 'count).', '<br', '/><br', '/>and', 't

In [23]:
TEXT.build_vocab(trainset, min_freq=5) # 단어 집합 생성
LABEL.build_vocab(trainset)


In [24]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 61663
클래스의 개수 : 2


In [25]:
print(TEXT.vocab.stoi)


defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f806bca8310>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'a': 3, 'and': 4, 'of': 5, 'to': 6, 'is': 7, 'in': 8, 'i': 9, 'this': 10, 'that': 11, 'it': 12, '/><br': 13, 'was': 14, 'as': 15, 'for': 16, 'with': 17, 'but': 18, 'on': 19, 'movie': 20, 'his': 21, 'are': 22, 'not': 23, 'you': 24, 'film': 25, 'have': 26, 'he': 27, 'be': 28, 'at': 29, 'one': 30, 'by': 31, 'an': 32, 'they': 33, 'from': 34, 'all': 35, 'who': 36, 'like': 37, 'so': 38, 'just': 39, 'or': 40, 'has': 41, 'about': 42, 'her': 43, "it's": 44, 'if': 45, 'some': 46, 'out': 47, 'what': 48, 'when': 49, 'very': 50, 'there': 51, 'more': 52, 'would': 53, 'even': 54, 'she': 55, 'good': 56, 'my': 57, 'only': 58, 'their': 59, 'no': 60, 'really': 61, 'had': 62, 'up': 63, 'can': 64, 'which': 65, 'see': 66, 'were': 67, 'than': 68, 'we': 69, '-': 70, 'been': 71, 'get': 72, 'will': 73, 'into': 74, 'much': 75, 'because': 76, 'story': 77, 'how': 78, 'm

In [26]:
trainset, valset = trainset.split(split_ratio=0.8)

In [27]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

In [28]:
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))

훈련 데이터의 미니 배치의 개수 : 500
테스트 데이터의 미니 배치의 개수 : 157
검증 데이터의 미니 배치의 개수 : 125


In [29]:
batch = next(iter(train_iter)) # 첫번째 미니배치
print(batch.text.shape)

torch.Size([64, 774])


In [30]:
batch = next(iter(train_iter)) # 두번째 미니배치
print(batch.text.shape)

torch.Size([64, 866])


In [31]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

In [32]:
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [33]:
model = GRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [34]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()


In [35]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [36]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))


In [1]:
!pip install transformers


     |████████████████████████████████| 2.5MB 7.9MB/s 
     |████████████████████████████████| 3.3MB 33.3MB/s 
     |████████████████████████████████| 901kB 37.6MB/s 


In [2]:
from transformers import pipeline
print(pipeline('sentiment-analysis')('we love you'))


[{'label': 'POSITIVE', 'score': 0.9998704791069031}]


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [5]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)